In [ ]:
import findspark
import folium

findspark.init()

from pyspark import sql, SparkConf, SparkContext

coordinates = (20.593684, 78.96288)
conf = SparkConf().setAppName("Read_CSV")
sc = SparkContext(conf=conf)
sqlContext = sql.SQLContext(sc)
map = folium.Map(location=coordinates, zoom_start=5)

dfSteden = sqlContext.read.csv("Steden.csv", header=True)
dfLucht = sqlContext.read.csv("Luchtvervuiling.csv", header=True)

In [ ]:
from folium.plugins import MarkerCluster

locations = dfSteden.select("lat", "lng", "city")
pollution = dfLucht.select("location", "so2", "no2")
index = 0
c = '#3186cc'
new = pollution.join(locations, pollution.location == locations.city, how='right')
new2 = new.filter("lat != ''")
joinDF = new2.filter("lng != ''")
joinDF = joinDF.dropDuplicates(['city']).collect()
marker_cluster = MarkerCluster().add_to(map)

for line in joinDF:
    if (joinDF[index][1] != 'NA') and (joinDF[index][1] is not None) and (float(joinDF[index][1]) > 4):
        c = '#ff0000'
    else:
        c = '#3186cc'
    folium.CircleMarker(
        location=[joinDF[index][3], joinDF[index][4]],
        radius=4,
        popup=joinDF[index][5],
        color=c,
        fill=True,
        fill_color='#3186cc'
    ).add_to(marker_cluster)
    index = index + 1

In [ ]:
map